In [17]:
# Importing the important libraries
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image

# Download the architecture of ResNet50 with ImageNet weights
base_model = ResNet50(include_top=False, weights='imagenet')
 
# Taking the output of the last convolution block in ResNet50
x = base_model.output
 
# Adding a Global Average Pooling layer
x = GlobalAveragePooling2D()(x)
 
# Adding a fully connected layer having 1024 neurons
x = Dense(1024, activation='relu')(x)
 
# Adding a fully connected layer having 2 neurons which will
# give the probability of image having either dog or cat
predictions = Dense(5, activation='softmax')(x)
 
# Model to be trained
model = Model(inputs=base_model.input, outputs=predictions)
 
# Training only top layers i.e. the layers which we have added in the end
for layer in base_model.layers:
    layer.trainable = False
 
# Compiling the model
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics = ['accuracy'], run_eagerly=True)
 
# Creating objects for image augmentations
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
 
test_datagen = ImageDataGenerator(rescale = 1./255)
 
# Proving the path of training and test dataset
# Setting the image input size as (224, 224)
# We are using class mode as binary because there are only two classes in our data
training_set = train_datagen.flow_from_directory('../../ANIMAL_DATASET/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
 
test_set = test_datagen.flow_from_directory('../../ANIMAL_DATASET/',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')
 
# Training the model for 5 epochs
model.fit(training_set, steps_per_epoch = len(training_set)//32, epochs = 5, validation_data = test_set, validation_steps = len(test_set)//32)
 
# We will try to train the last stage of ResNet50
for layer in base_model.layers[0:143]:
  layer.trainable = False
 
for layer in base_model.layers[143:]:
  layer.trainable = True
 
# Training the model for 10 epochs
model.fit(training_set,
                         steps_per_epoch = len(training_set)//32,
                         epochs = 10,
                         validation_data = test_set,
                         validation_steps = len(test_set)//32)
 
# Saving the weights in the current directory
model.save_weights("resnet50_weights.h5")
 
# Predicting the final result of image

test_image = image.load_img('../../ANIMAL_DATASET/Jaguar/animal-2607__340.jpg', target_size = (224, 224))
test_image = image.img_to_array(test_image)
 
# Expanding the 3-d image to 4-d image.
# The dimensions will be Batch, Height, Width, Channel
test_image = np.expand_dims(test_image, axis = 0)
 
# Predicting the final class
result = model.predict(test_image)[0].argmax()
 
# Fetching the class labels
labels = training_set.class_indices
labels = list(labels.items())
 
# Printing the final label
for label, i in labels:
    if i == result:
        print("The test image has: ", label)
        break

Found 170 images belonging to 5 classes.
Found 170 images belonging to 5 classes.
Epoch 1/5


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [4]:
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np


model = ResNet50()

In [13]:
image1 = image.load_img("../../IN_YOUR_FACE_DATASET/UNMASKED_CELEBRITIES/Angelina_Jolie/Angelina_Jolie_015.jpg", target_size = (224, 224))

transformedImage = image.img_to_array(image1)
transformedImage = np.expand_dims(transformedImage, axis = 0)
transformedImage = preprocess_input(transformedImage)

prediction = model.predict(transformedImage)

predictionLabel = decode_predictions(prediction, top = 1)

print('%s (%.2f%%)' % (predictionLabel[0][0][1], predictionLabel[0][0][2]*100 ))

1/1 [==============================] - 0s 27ms/step
miniskirt (40.28%)


In [20]:
import face_recognition
import cv2
import numpy as np

video_capture = cv2.VideoCapture("../../Dataset/Video1.mp4")



# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    ret, frame = video_capture.read()

    if process_this_frame:
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        rgb_small_frame = small_frame[:, :, ::-1]
        
        ###
        transformedImage = image.img_to_array(rgb_small_frame)
        transformedImage = np.expand_dims(transformedImage, axis = 0)
        transformedImage = preprocess_input(transformedImage)

        prediction = model.predict(transformedImage)

        predictionLabel = decode_predictions(prediction, top = 1)
        ###

        face_locations = face_recognition.face_locations(rgb_small_frame)

    #process_this_frame = not process_this_frame


    for (top, right, bottom, left) in face_locations:
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        cv2.rectangle(frame, (left - 10, top - 50), (right + 10, bottom + 50), (0, 0, 255), 2)
        cv2.rectangle(frame, (left-11, bottom + 50), (right + 11, bottom + 75), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, predictionLabel[0][0][1], (left - 6, bottom + 65), font, 0.5, (255, 255, 255), 1)

    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 24ms/step
